##### In this we will learn how to access a webpage and get the data into our local work environment

In [1]:
# required python packages
import pandas as pd
# we work with stocks data in a table format
import numpy as np
import re
import requests
# in order to send request to webpage
from bs4 import BeautifulSoup
# to fetch the data
from datetime import datetime
# for saving the file to excel
from pandas import ExcelWriter

##### Here we try to access NSE India page and try to fetch the data tables for stocks that we need 

I've collected some stock symbols from NSE website and stored them in excel file(symbols.xlsx)

In [2]:
symbols_dataframe = pd.read_excel('symbols.xlsx')

In [3]:
symbols_dataframe.head()

,S. No,Name of the Stock,Symbol of the Stock,URL
0,1,ACC LIMITED,ACC,https://www.nseindia.com/live_market/dynaConte...
1,2,ADANI ENTERPRISES LIMITED,ADANIENT,https://www.nseindia.com/live_market/dynaConte...
2,3,ADANI PORT & SEZ LTD,ADANIPORTS,https://www.nseindia.com/live_market/dynaConte...
3,4,ADANI POWER LTD,ADANIPOWER,https://www.nseindia.com/live_market/dynaConte...
4,5,ADITYA BIRLA NUVO LIMITED,ABIRLANUVO,https://www.nseindia.com/live_market/dynaConte...


In [4]:
list_of_symbols = list(symbols_dataframe['Symbol of the Stock'])

In [5]:
list_of_symbols[:5]

['ACC', 'ADANIENT', 'ADANIPORTS', 'ADANIPOWER', 'ABIRLANUVO']

In [6]:
list_of_urls = list(symbols_dataframe['URL'])

In [7]:
list_of_urls[:5]

['https://www.nseindia.com/live_market/dynaContent/live_watch/option_chain/optionKeys.jsp?symbolCode=209&symbol=ACC&symbol=acc&instrument=OPTSTK&date=-&segmentLink=17&segmentLink=17',
 'https://www.nseindia.com/live_market/dynaContent/live_watch/option_chain/optionKeys.jsp?symbolCode=424&symbol=ADANIENT&symbol=ADANIENT&instrument=OPTSTK&date=-&segmentLink=17&segmentLink=17',
 'https://www.nseindia.com/live_market/dynaContent/live_watch/option_chain/optionKeys.jsp?symbolCode=2683&symbol=ADANIPORTS&symbol=ADANIPORTS&instrument=OPTSTK&date=-&segmentLink=17&segmentLink=17',
 'https://www.nseindia.com/live_market/dynaContent/live_watch/option_chain/optionKeys.jsp?symbolCode=2901&symbol=ADANIPOWER&symbol=ADANIPOWER&instrument=OPTSTK&date=-&segmentLink=17&segmentLink=17',
 'https://www.nseindia.com/live_market/dynaContent/live_watch/option_chain/optionKeys.jsp?symbolCode=219&symbol=ABIRLANUVO&symbol=ABIRLANUVO&instrument=OPTSTK&date=-&segmentLink=17&segmentLink=17']

##### I'll try to explain with a single stock and later we shall iterate the logic over all the stocks

In [8]:
# Example, we are taking HCL Technologies
url = list_of_urls[77]

In [9]:
url

'https://www.nseindia.com/live_market/dynaContent/live_watch/option_chain/optionKeys.jsp?symbolCode=1828&symbol=HCLTECH&symbol=HCLTECH&instrument=OPTSTK&date=-&segmentLink=17&segmentLink=17'

In [10]:
# The following line connects through your default system proxy settings
response = requests.get(url)

In [11]:
# Code of the response indicates the status 
response

<Response [200]>

In [12]:
# This gives the whole HTML code of the page
html_content = response.content

In [13]:
html_content

b'\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">\r\n<html xmlns="http://www.w3.org/1999/xhtml">\r\n<head>\r\n<!-- Global site tag (gtag.js) - Google Analytics -->\r\n<script async src="https://www.googletagmanager.com/gtag/js?id=UA-108453261-1"></script>\r\n<script>\r\n  window.dataLayer = window.dataLayer || [];\r\n  function gtag(){dataLayer.push(arguments);}\r\n  gtag(\'js\', new Date());\r\n\r\n  gtag(\'config\', \'UA-108453261-1\');\r\n</script>\r\n<meta http-equiv="X-UA-Compatible" content="IE=8" />\r\n\r\n\r\n<meta http-equiv="Content-Type" content="text/html; charset=utf-8" />\r\n<title>NSE - National Stock Exchange of India Ltd.</title>\r\n<script type="text/javascript">\r\nvar page=["livewth_optch","liveMarket"];\r\n</script>\r\n\r\n<script type="text/javascript" src="/common/js/jquery-1.4.4.min.js"></script>\r\n<script type="text/javascript" src="/common/js/

In [14]:
# This BeautifulSoup gives the proper HTML format of the page
soup = BeautifulSoup(html_content, 'html.parser')

In [15]:
soup


<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">

<html xmlns="http://www.w3.org/1999/xhtml">
<head>
<!-- Global site tag (gtag.js) - Google Analytics -->
<script async="" src="https://www.googletagmanager.com/gtag/js?id=UA-108453261-1"></script>
<script>
  window.dataLayer = window.dataLayer || [];
  function gtag(){dataLayer.push(arguments);}
  gtag('js', new Date());

  gtag('config', 'UA-108453261-1');
</script>
<meta content="IE=8" http-equiv="X-UA-Compatible"/>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<title>NSE - National Stock Exchange of India Ltd.</title>
<script type="text/javascript">
var page=["livewth_optch","liveMarket"];
</script>
<script src="/common/js/jquery-1.4.4.min.js" type="text/javascript"></script>
<script src="/common/js/mainNav.js" type="text/javascript"></script>
<script src="/common/js/js.js" type="text/javascript"></script>
<!--date util -->
<script src="/

In [16]:
# Now the code looks quite simple to fetch whatever the data that is needed and we extract a table by the id 'octable'
table = soup.find(id ='octable')

In [17]:
table

<table border="0" cellpadding="0" cellspacing="0" id="octable" width="100%">
<thead>
<tr>
<th colspan="11">CALLS</th>
<th> </th>
<th colspan="11">PUTS</th>
</tr>
<tr>
<!-- <th>Quote </th>   -->
<th title="Chart">Chart</th>
<th title="Open Interest">OI</th>
<th title="Change in Open Interest">Chng in OI</th>
<th title="Traded Volume">Volume</th>
<th title="Implied Volatility">IV</th>
<th title="Last Traded Price">LTP</th>
<!--**-->
<th title="Net Change">Net Chng</th>
<th title="Bid Quantity">Bid<br/>Qty</th>
<th title="Bid Price">Bid<br/>Price</th>
<th title="Ask Price">Ask<br/>Price</th>
<th title="Ask Quantity">Ask<br/>Qty</th>
<th title="Strike Price">Strike Price</th>
<th title="Bid Quantity">Bid<br/>Qty</th>
<th title="Bid Price">Bid<br/>Price</th>
<th title="Ask Price">Ask<br/>Price</th>
<th title="Ask Quantity">Ask<br/>Qty</th>
<th title="Net Change">Net Chng</th>
<th title="Last Traded Price">LTP</th>
<th title="Implied Volatility">IV</th>
<th title="Traded Volume">Volume</th>


In [18]:
# for date and initial details
upper_table = soup.find(name='table')

In [19]:
upper_table

<table width="100%"><tr><td>
<div class="tphead"><h2>Option Chain (Equity Derivatives)</h2></div>
</td><td align="right">
<div style="float:right; font-size:1.2em;">
<span>Underlying Stock: <b style="font-size:1.2em;">HCLTECH 886.05</b> </span>
<span>As on Dec 26, 2017 15:30:30 IST<a> <img onclick="refresh();" src="/live_market/resources/images/refressbtn.gif" style="cursor: pointer" title="refresh"/></a></span></div>
</td></tr></table>

In [20]:
upper_table.find('h2').text.strip()

'Option Chain (Equity Derivatives)'

In [21]:
(upper_table.find_all('span'))[0].text.strip()

'Underlying Stock: HCLTECH 886.05'

In [22]:
# For row data
table.find_all('tr')

[<tr>
 <th colspan="11">CALLS</th>
 <th> </th>
 <th colspan="11">PUTS</th>
 </tr>, <tr>
 <!-- <th>Quote </th>   -->
 <th title="Chart">Chart</th>
 <th title="Open Interest">OI</th>
 <th title="Change in Open Interest">Chng in OI</th>
 <th title="Traded Volume">Volume</th>
 <th title="Implied Volatility">IV</th>
 <th title="Last Traded Price">LTP</th>
 <!--**-->
 <th title="Net Change">Net Chng</th>
 <th title="Bid Quantity">Bid<br/>Qty</th>
 <th title="Bid Price">Bid<br/>Price</th>
 <th title="Ask Price">Ask<br/>Price</th>
 <th title="Ask Quantity">Ask<br/>Qty</th>
 <th title="Strike Price">Strike Price</th>
 <th title="Bid Quantity">Bid<br/>Qty</th>
 <th title="Bid Price">Bid<br/>Price</th>
 <th title="Ask Price">Ask<br/>Price</th>
 <th title="Ask Quantity">Ask<br/>Qty</th>
 <th title="Net Change">Net Chng</th>
 <th title="Last Traded Price">LTP</th>
 <th title="Implied Volatility">IV</th>
 <th title="Traded Volume">Volume</th>
 <th title="Change in Open Interest">Chng in OI</th>
 <th

In [23]:
table.find_all('tr')[2].find_all('td')

[<td><a href="javascript:chartPopup('HCLTECH', 'OPTSTK', '28DEC2017', '760.00','CE','');"><img alt="Graph" src="/live_market/resources/images/grficon.gif"/></a></td>,
 <td class="ylwbg">-</td>,
 <td class="ylwbg">-</td>,
 <td class="ylwbg">-</td>,
 <td class="ylwbg">-</td>,
 <td class="ylwbg">
 							
 							-
 							
 							</td>,
 <td class="ylwbg" style="color:Black;">-</td>,
 <td class="ylwbg"> 1,400</td>,
 <td class="ylwbg"> 122.50</td>,
 <td class="ylwbg"> 131.20</td>,
 <td class="ylwbg"> 1,400</td>,
 <td class="grybg"><a href="/live_market/dynaContent/live_watch/option_chain/optionDates.jsp?symbol=HCLTECH&amp;instrument=OPTSTK&amp;strike=760.00"><b>760.00</b></a></td>,
 <td class="nobg"> 700</td>,
 <td class="nobg"> 0.05</td>,
 <td class="nobg"> 0.50</td>,
 <td class="nobg"> 700</td>,
 <td class="nobg" style="color:Black;">-</td>,
 <td class="nobg">
 <!-- <a href="javascript:popup1('HCLTECH','OPTSTK','28DEC2017','760.00','PE')"> 0.20</a> -->
 <a href="/live_market/dynaConten

In [24]:
data = []
rows = table.find_all('tr')
for row in rows[1:]:
    cols = row.find_all('th')
    if len(cols) == 0:
        cols = row.find_all('td')
        cols = [ele.text.strip() for ele in cols]
        
        data.append([ele for ele in cols if ele])
        
    else:
        cols = [ele.text.strip() for ele in cols]
        data.append([ele for ele in cols[1:-1] if ele])


In [25]:
data

[['OI',
  'Chng in OI',
  'Volume',
  'IV',
  'LTP',
  'Net Chng',
  'BidQty',
  'BidPrice',
  'AskPrice',
  'AskQty',
  'Strike Price',
  'BidQty',
  'BidPrice',
  'AskPrice',
  'AskQty',
  'Net Chng',
  'LTP',
  'IV',
  'Volume',
  'Chng in OI',
  'OI'],
 ['-',
  '-',
  '-',
  '-',
  '-',
  '-',
  '1,400',
  '122.50',
  '131.20',
  '1,400',
  '760.00',
  '700',
  '0.05',
  '0.50',
  '700',
  '-',
  '0.20',
  '-',
  '-',
  '-',
  '14,700'],
 ['700',
  '-',
  '-',
  '-',
  '109.50',
  '-',
  '3,500',
  '104.15',
  '111.90',
  '3,500',
  '780.00',
  '-',
  '-',
  '1.50',
  '1,400',
  '-0.15',
  '0.35',
  '83.17',
  '1',
  '-700',
  '8,400'],
 ['1,400',
  '-',
  '-',
  '-',
  '73.00',
  '-',
  '2,800',
  '81.90',
  '92.20',
  '2,800',
  '800.00',
  '7,000',
  '0.05',
  '0.20',
  '2,100',
  '-0.05',
  '0.20',
  '-',
  '-',
  '-',
  '22,400'],
 ['8,400',
  '-',
  '-',
  '-',
  '52.20',
  '-',
  '3,500',
  '61.45',
  '71.40',
  '2,800',
  '820.00',
  '2,100',
  '0.20',
  '0.45',
  '700',
  

In [26]:
pd.DataFrame(data[1:-1], columns=data[0])

,OI,Chng in OI,Volume,IV,LTP,Net Chng,BidQty,BidPrice,AskPrice,AskQty,...,BidQty,BidPrice,AskPrice,AskQty,Net Chng,LTP,IV,Volume,Chng in OI,OI
0,-,-,-,-,-,-,"1,400",122.50,131.20,"1,400",...,700,0.05,0.50,700,-,0.20,-,-,-,"14,700"
1,700,-,-,-,109.50,-,"3,500",104.15,111.90,"3,500",...,-,-,1.50,"1,400",-0.15,0.35,83.17,1,-700,"8,400"
2,"1,400",-,-,-,73.00,-,"2,800",81.90,92.20,"2,800",...,"7,000",0.05,0.20,"2,100",-0.05,0.20,-,-,-,"22,400"
3,"8,400",-,-,-,52.20,-,"3,500",61.45,71.40,"2,800",...,"2,100",0.20,0.45,700,-0.20,0.20,50.02,4,700,"32,900"
4,"33,600","-1,400",2,65.05,49.25,-0.75,"2,800",42.15,53.70,"3,500",...,700,0.20,0.35,"3,500",-0.35,0.20,36.50,47,"-17,500","100,800"
5,"64,400","-1,400",11,32.46,27.55,5.55,700,25.20,27.70,700,...,700,0.30,0.45,"4,200",-0.90,0.45,26.53,100,"-11,200","135,100"
6,"139,300","-4,200",266,21.86,9.55,-2.25,"7,000",9.00,10.20,700,...,700,2.90,3.35,700,-1.40,3.20,22.63,268,"12,600","85,400"
7,"361,200","88,200",761,29.83,2.90,-0.65,700,2.50,3.20,"1,400",...,"2,100",14.10,16.75,"1,400",-0.95,14.60,20.50,9,"-1,400","12,600"
8,"72,100","-8,400",285,36.96,0.95,-0.35,"3,500",0.95,1.00,700,...,"3,500",30.30,37.70,"3,500",-6.20,31.00,-,1,-700,"3,500"
9,"46,900","-2,100",11,46.14,0.55,-0.15,"1,400",0.50,1.00,"1,400",...,"2,100",50.70,58.45,"4,900",-,-,-,-,-,-


### That's it ! We can save it as excel to our local machine

In [27]:
count = 0
# for symbol, url in zip(list_of_symbols, list_of_urls): 
# I'm commenting the abpve line because I don't want to save all that data, but if you want, you can use that
for symbol, url in zip(list_of_symbols[1:2], list_of_urls[1:2]):
    count += 1
    writer = None
    try :
        response = requests.get(url)
        html_content = response.content
        soup = BeautifulSoup(html_content, 'html.parser')
        up_table = soup.find(name='table')
        upper_line = []
        upper_line.append(up_table.find('h2').text.strip())
        for i in range(15):
            upper_line.append(' ')
        upper_line.append((up_table.find_all('span'))[0].text.strip())
        upper_line.append((up_table.find_all('span'))[1].text.strip())
        table = soup.find(id ='octable')
        rows = table.find_all('tr')
        data = []
        data.append([' ']*10 + [symbol] + [' ']*15)
        data.append(upper_line)
        data.append([' ']*4 + ['CALLS'] + [' ']*11 + ['PUTS'] + [' ']*4)
        for row in rows[1:-1]:
            cols = row.find_all('th')
            if len(cols) == 0:
                cols = row.find_all('td')
                cols = [ele.text.strip() for ele in cols]

                data.append([ele for ele in cols if ele])

            else:
                cols = [ele.text.strip() for ele in cols]
                data.append([ele for ele in cols[1:-1] if ele])
        total_row = rows[-1].find_all('td')
        calls = [ele.text.strip() for ele in total_row[1:4]]
        puts = [ele.text.strip() for ele in total_row[-4:-1]]
        data.append(calls + ['<-Total'] + [' ']*13 + ['Total->'] + puts)
#         print(data[3])
        writer = ExcelWriter(datetime.today().date().strftime ("%d-%m-%Y")+"_"+symbol+'.xlsx')
        pd.DataFrame(data).to_excel(writer, symbol)
        writer.save()
        print(str(count) +". EXCEL file is created for "+ symbol)
    except PermissionError:
        print(str(count) +". Make sure you close the previously opened Excel file of "+ symbol)
        continue
    except Exception as e:
        print(str(count) +". Check your internet connection", e)
        continue

1. EXCEL file is created for ADANIENT


#### QUICK TIP

Extracting data from excel file looks simple, but it takes long time when you give a GB of file

If you intend to use this list for multiple times, what you can do is to make use of numpy module

In [28]:
np.save('list_of_symbols.npy', list_of_symbols)
np.save('list_of_urls.npy', list_of_urls)

In [29]:
# You can directly extract them like this
list_of_symbols = np.load('list_of_symbols.npy').tolist()
list_of_urls = np.load('list_of_urls.npy').tolist()

 You can save dictionaries as well